In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
from fastai.text.all import *
from reformer_fastai.reformer import LSHLM
from reformer_fastai.data import TwinSequence, MaskTargCallback
from reformer_fastai.metrics import MaskedAccuracy

In [ ]:
#hide
#skip
torch.cuda.set_device(7)

In [ ]:
#all_slow

# Minimal training example

Sequence length of 1024, similar what is used in the paper.

In [ ]:
bs, sl = 32,64
n_epochs = 1
train_sz = 500
valid_sz = 100
dls = DataLoaders.from_dsets(TwinSequence(sl, train_sz), TwinSequence(sl, valid_sz), bs=bs, shuffle=False, device='cuda')

In [ ]:
n_hashes=4
bucket_size = 32
assert sl % (bucket_size * 2) == 0

In [ ]:
model = model = LSHLM(vocab_sz=128, d_model=256, n_layers=1, n_heads=4, max_seq_len=sl,
              bucket_size=bucket_size, n_hashes=n_hashes, causal=True, 
                      use_lsh=True)
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(ignore_index=-100), 
                metrics=MaskedAccuracy(), cbs=[MaskTargCallback()])
learn.fit_one_cycle(n_epochs, 1e-3)

epoch,train_loss,valid_loss,masked_accuracy,time
0,4.812356,4.760940,0.038438,00:03
